In [2]:
!pip install supabase tf2onnx prophet joblib

  Using cached supabase-2.15.3-py3-none-any.whl.metadata (11 kB)
  Using cached tf2onnx-1.16.1-py3-none-any.whl.metadata (1.3 kB)
  Using cached gotrue-2.12.0-py3-none-any.whl.metadata (6.1 kB)
  Using cached postgrest-1.0.2-py3-none-any.whl.metadata (3.5 kB)
  Using cached realtime-2.4.3-py3-none-any.whl.metadata (6.7 kB)
  Using cached storage3-0.11.3-py3-none-any.whl.metadata (1.8 kB)
  Using cached supafunc-0.9.4-py3-none-any.whl.metadata (1.2 kB)
  Using cached pytest_mock-3.14.1-py3-none-any.whl.metadata (3.9 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 455.8/455.8 kB 10.2 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.9/169.9 kB 8.7 MB/s eta 0:00:00
  Attempting uninstall: websockets
    Found existing installation: websockets 15.0.1
    Uninstalling websockets-15.0.1:
      Successfully uninstalled websockets-15.0.1
ERROR: pip's dependency resolver does not currently take int

In [3]:
from keras.models import Sequential
from keras.layers import SimpleRNN, LSTM, GRU, Dense
from sklearn.preprocessing import MinMaxScaler
import pickle
from supabase import create_client, Client
import pandas as pd
import numpy as np
from datetime import datetime, timedelta, timezone
import plotly.express as px
import tensorflow as tf
import tf2onnx
from prophet import Prophet
import joblib

2025-06-19 14:26:40.355862: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750343200.590777      69 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750343200.660849      69 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
supabase: Client = create_client(SUPABASE_URL, SUPABASE_KEY)

In [5]:
# FUNCION DE DESCARGA DE DATOS 
def get_demanda_data(page_size=1000):
    end = datetime.now(timezone.utc)
    start = end - timedelta(days=3650)
    all_data = []
    offset = 0
    while True:
        response = (
            supabase.table("demanda")
            .select("datetime,value")
            .gte("datetime", start.isoformat())
            .lte("datetime", end.isoformat())
            .range(offset, offset + page_size - 1)
            .execute()
        )
        data = response.data
        if not data:
            break
        all_data.extend(data)
        offset += page_size
        if len(data) < page_size:
            break

    if not all_data:
        return pd.DataFrame()
    df = pd.DataFrame(all_data)
    df["datetime"] = pd.to_datetime(df["datetime"])
    return df

# FUNCION DE ESCALADO Y GUARDADO DEL SCALER
def escalar_datos(df, save_path='scaler.pkl'):
    scaler = MinMaxScaler()
    df['value_scaled'] = scaler.fit_transform(df[['value']])

    with open(save_path, 'wb') as f:
        pickle.dump(scaler, f)

    print(f" Datos escalados y scaler guardado en '{save_path}'.")
    return df, scaler

# FUNCION PARA DIVIDIR EN TRAIN Y TEST
def dividir_train_test(df, test_size=0.2):
    df = df.copy()
    df = df.set_index('datetime')
    split_point = int(len(df) * (1 - test_size))
    df_train = df.iloc[:split_point]
    df_test = df.iloc[split_point:]
    return df_train, df_test


In [6]:
# Cargar datos
df = get_demanda_data()
df, scaler = escalar_datos(df)
# Guardar dataset como CSV para usarlo en Streamlit
df.to_csv('datos_prediccion.csv', index=False)
print("✅ Dataset guardado como 'datos_prediccion.csv'")
df_train, df_test = dividir_train_test(df)

def crear_secuencias(datos, n_pasos):
    X, y = [], []
    for i in range(len(datos) - n_pasos):
        X.append(datos[i:i + n_pasos])
        y.append(datos[i + n_pasos])
    return np.array(X), np.array(y)

 Datos escalados y scaler guardado en 'scaler.pkl'.
✅ Dataset guardado como 'datos_prediccion.csv'


In [7]:
# Configuración de hiperparámetros
model_type = 'SimpleRNN'  # Cambia por 'SimpleRNN', 'LSTM' o 'GRU' para entrenar otros modelos
loss_function = 'mse'  # Puedes cambiar por 'mse' o 'mae'
n_pasos = 24
n_epochs = 40
batch_size = 32
unidades = 50

# Crear secuencias
X_train, y_train = crear_secuencias(df_train['value_scaled'].values, n_pasos)
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))

# Crear secuencias para Test
X_test, y_test = crear_secuencias(df_test['value_scaled'].values, n_pasos)
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

# Definir el modelo
if model_type == 'SimpleRNN':
    rnn_layer = SimpleRNN(unidades, activation='tanh', input_shape=(n_pasos, 1))
elif model_type == 'LSTM':
    rnn_layer = LSTM(unidades, activation='tanh', input_shape=(n_pasos, 1))
elif model_type == 'GRU':
    rnn_layer = GRU(unidades, activation='tanh', input_shape=(n_pasos, 1))

model = Sequential([
    rnn_layer,
    Dense(1)
])

model.compile(optimizer='adam', loss=loss_function)

# Entrenamiento
history = model.fit(
    X_train, y_train,
    epochs=n_epochs,
    batch_size=batch_size,
    validation_data=(X_test, y_test),  # ⚙️ Esto es obligatorio para tener val_loss
    verbose=1
)

# Guardar el scaler
#model.save(f'{model_type}_model_{loss_function}.keras')

with open(f'scaler_{model_type}_{loss_function}.pkl', 'wb') as f:
    pickle.dump(scaler, f)
    
# Guardar el history
with open(f"{model_type}_history_{loss_function}.pkl", 'wb') as f:
    pickle.dump(history.history, f)

# Guardar el modelo en ONNX
# 1. Crear el input_signature
input_signature = [tf.TensorSpec([None, n_pasos, 1], tf.float32, name="input")]

# 2. Decorar la función (no obtener concrete_func)
@tf.function(input_signature=input_signature)
def model_func(x):
    return model(x)

# 3. Convertir a ONNX directamente desde la función decorada
onnx_model, _ = tf2onnx.convert.from_function(
    model_func,  # 👉 Aquí debes pasar la función decorada, no la concrete_func
    input_signature=input_signature,
    opset=13,
    output_path=f'{model_type}_model_{loss_function}.onnx'
)

print(f'✅ Modelo {model_type}_model_{loss_function}.onnx entrenado y guardado correctamente.')
print(f'Scaler {model_type} guardado.')
print(f"Historial de entrenamiento {model_type} guardado.")

Epoch 1/40


2025-06-19 14:27:22.105905: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


2188/2188 ━━━━━━━━━━━━━━━━━━━━ 16s 6ms/step - loss: 0.0226 - val_loss: 5.8308e-04
Epoch 2/40
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 13s 6ms/step - loss: 4.9697e-04 - val_loss: 3.0702e-04
Epoch 3/40
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 13s 6ms/step - loss: 3.0384e-04 - val_loss: 2.2155e-04
Epoch 4/40
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 14s 6ms/step - loss: 2.0378e-04 - val_loss: 1.4076e-04
Epoch 5/40
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 13s 6ms/step - loss: 1.9443e-04 - val_loss: 1.7034e-04
Epoch 6/40
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 14s 6ms/step - loss: 1.8654e-04 - val_loss: 1.2677e-04
Epoch 7/40
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 13s 6ms/step - loss: 1.8487e-04 - val_loss: 1.4368e-04
Epoch 8/40
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 13s 6ms/step - loss: 1.7096e-04 - val_loss: 1.3066e-04
Epoch 9/40
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 14s 6ms/step - loss: 1.6307e-04 - val_loss: 1.2545e-04
Epoch 10/40
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 13s 6ms/step - loss: 1.5927e-04 - val_loss: 1.3518e-04
Epoch 11/40
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 1

I0000 00:00:1750343776.978634      69 devices.cc:67] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
I0000 00:00:1750343776.978852      69 single_machine.cc:361] Starting new session
I0000 00:00:1750343777.186003      69 devices.cc:67] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
I0000 00:00:1750343777.186292      69 single_machine.cc:361] Starting new session
I0000 00:00:1750343777.300864      69 mlir_graph_optimization_pass.cc:401] MLIR V1 optimization pass is not enabled


✅ Modelo SimpleRNN_model_mse.onnx entrenado y guardado correctamente.
Scaler SimpleRNN guardado.
Historial de entrenamiento SimpleRNN guardado.


In [8]:
# Gráfico de pérdida
fig_loss = px.line(y=history.history['loss'], title='Pérdida durante el entrenamiento')
fig_loss.show()

In [9]:
# Configuración de hiperparámetros
model_type = 'LSTM'  # Cambia por 'SimpleRNN', 'LSTM' o 'GRU' para entrenar otros modelos
loss_function = 'mse'  # Puedes cambiar por 'mse' o 'mae'
n_pasos = 24
n_epochs = 40
batch_size = 32
unidades = 50

# Crear secuencias
X_train, y_train = crear_secuencias(df_train['value_scaled'].values, n_pasos)
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))

# Crear secuencias para Test
X_test, y_test = crear_secuencias(df_test['value_scaled'].values, n_pasos)
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

# Definir el modelo
if model_type == 'SimpleRNN':
    rnn_layer = SimpleRNN(unidades, activation='tanh', input_shape=(n_pasos, 1))
elif model_type == 'LSTM':
    rnn_layer = LSTM(unidades, activation='tanh', input_shape=(n_pasos, 1))
elif model_type == 'GRU':
    rnn_layer = GRU(unidades, activation='tanh', input_shape=(n_pasos, 1))

model = Sequential([
    rnn_layer,
    Dense(1)
])

model.compile(optimizer='adam', loss=loss_function)

# Entrenamiento
history = model.fit(
    X_train, y_train,
    epochs=n_epochs,
    batch_size=batch_size,
    validation_data=(X_test, y_test),  # ⚙️ Esto es obligatorio para tener val_loss
    verbose=1
)

# Guardar el scaler
#model.save(f'{model_type}_model_{loss_function}.keras')

with open(f'scaler_{model_type}_{loss_function}.pkl', 'wb') as f:
    pickle.dump(scaler, f)
    
# Guardar el history
with open(f"{model_type}_history_{loss_function}.pkl", 'wb') as f:
    pickle.dump(history.history, f)

# Guardar el modelo en ONNX
# 1. Crear el input_signature
input_signature = [tf.TensorSpec([None, n_pasos, 1], tf.float32, name="input")]

# 2. Decorar la función (no obtener concrete_func)
@tf.function(input_signature=input_signature)
def model_func(x):
    return model(x)

# 3. Convertir a ONNX directamente desde la función decorada
onnx_model, _ = tf2onnx.convert.from_function(
    model_func,  # 👉 Aquí debes pasar la función decorada, no la concrete_func
    input_signature=input_signature,
    opset=13,
    output_path=f'{model_type}_model_{loss_function}.onnx'
)

print(f'✅ Modelo {model_type}_model_{loss_function}.onnx entrenado y guardado correctamente.')
print(f'Scaler {model_type} guardado.')
print(f"Historial de entrenamiento {model_type} guardado.")

Epoch 1/40


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



2188/2188 ━━━━━━━━━━━━━━━━━━━━ 29s 12ms/step - loss: 0.0218 - val_loss: 8.6869e-04
Epoch 2/40
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 26s 12ms/step - loss: 6.5715e-04 - val_loss: 5.0085e-04
Epoch 3/40
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 26s 12ms/step - loss: 3.9133e-04 - val_loss: 3.0920e-04
Epoch 4/40
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 26s 12ms/step - loss: 2.5886e-04 - val_loss: 2.1993e-04
Epoch 5/40
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 26s 12ms/step - loss: 2.3139e-04 - val_loss: 1.9392e-04
Epoch 6/40
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 26s 12ms/step - loss: 2.1409e-04 - val_loss: 2.9672e-04
Epoch 7/40
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 26s 12ms/step - loss: 2.0373e-04 - val_loss: 1.9941e-04
Epoch 8/40
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 26s 12ms/step - loss: 1.8498e-04 - val_loss: 2.0042e-04
Epoch 9/40
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 26s 12ms/step - loss: 1.6774e-04 - val_loss: 1.9424e-04
Epoch 10/40
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 26s 12ms/step - loss: 1.5517e-04 - val_loss: 2.5243e-04
Epoch 11/40
2188/2188 ━━━━━━━━━━━━

I0000 00:00:1750344828.262447      69 devices.cc:67] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
I0000 00:00:1750344828.262662      69 single_machine.cc:361] Starting new session
I0000 00:00:1750344828.490053      69 devices.cc:67] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
I0000 00:00:1750344828.490250      69 single_machine.cc:361] Starting new session


✅ Modelo LSTM_model_mse.onnx entrenado y guardado correctamente.
Scaler LSTM guardado.
Historial de entrenamiento LSTM guardado.


In [10]:
px.line(y=history.history['loss'], title='Pérdida durante el entrenamiento').show()

In [11]:
# Configuración de hiperparámetros
model_type = 'GRU'  # Cambia por 'SimpleRNN', 'LSTM' o 'GRU' para entrenar otros modelos
loss_function = 'mse'  # Puedes cambiar por 'mse' o 'mae'
n_pasos = 24
n_epochs = 40
batch_size = 32
unidades = 50

# Crear secuencias
X_train, y_train = crear_secuencias(df_train['value_scaled'].values, n_pasos)
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))

# Crear secuencias para Test
X_test, y_test = crear_secuencias(df_test['value_scaled'].values, n_pasos)
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

# Definir el modelo
if model_type == 'SimpleRNN':
    rnn_layer = SimpleRNN(unidades, activation='tanh', input_shape=(n_pasos, 1))
elif model_type == 'LSTM':
    rnn_layer = LSTM(unidades, activation='tanh', input_shape=(n_pasos, 1))
elif model_type == 'GRU':
    rnn_layer = GRU(unidades, activation='tanh', input_shape=(n_pasos, 1))

model = Sequential([
    rnn_layer,
    Dense(1)
])

model.compile(optimizer='adam', loss=loss_function)

# Entrenamiento
history = model.fit(
    X_train, y_train,
    epochs=n_epochs,
    batch_size=batch_size,
    validation_data=(X_test, y_test),  # ⚙️ Esto es obligatorio para tener val_loss
    verbose=1
)

# Guardar el scaler
#model.save(f'{model_type}_model_{loss_function}.keras')

with open(f'scaler_{model_type}_{loss_function}.pkl', 'wb') as f:
    pickle.dump(scaler, f)
    
# Guardar el history
with open(f"{model_type}_history_{loss_function}.pkl", 'wb') as f:
    pickle.dump(history.history, f)

# Guardar el modelo en ONNX
# 1. Crear el input_signature
input_signature = [tf.TensorSpec([None, n_pasos, 1], tf.float32, name="input")]

# 2. Decorar la función (no obtener concrete_func)
@tf.function(input_signature=input_signature)
def model_func(x):
    return model(x)

# 3. Convertir a ONNX directamente desde la función decorada
onnx_model, _ = tf2onnx.convert.from_function(
    model_func,  # 👉 Aquí debes pasar la función decorada, no la concrete_func
    input_signature=input_signature,
    opset=13,
    output_path=f'{model_type}_model_{loss_function}.onnx'
)

print(f'✅ Modelo {model_type}_model_{loss_function}.onnx entrenado y guardado correctamente.')
print(f'Scaler {model_type} guardado.')
print(f"Historial de entrenamiento {model_type} guardado.")

Epoch 1/40


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



2188/2188 ━━━━━━━━━━━━━━━━━━━━ 32s 14ms/step - loss: 0.0087 - val_loss: 5.1514e-04
Epoch 2/40
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 29s 13ms/step - loss: 4.4948e-04 - val_loss: 3.1484e-04
Epoch 3/40
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 29s 13ms/step - loss: 3.0965e-04 - val_loss: 3.4783e-04
Epoch 4/40
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 29s 13ms/step - loss: 2.8716e-04 - val_loss: 2.3463e-04
Epoch 5/40
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 29s 13ms/step - loss: 2.4431e-04 - val_loss: 1.8350e-04
Epoch 6/40
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 29s 13ms/step - loss: 2.1917e-04 - val_loss: 1.8782e-04
Epoch 7/40
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 29s 13ms/step - loss: 1.9379e-04 - val_loss: 1.5869e-04
Epoch 8/40
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 29s 13ms/step - loss: 1.8557e-04 - val_loss: 1.3779e-04
Epoch 9/40
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 29s 13ms/step - loss: 1.7760e-04 - val_loss: 1.4580e-04
Epoch 10/40
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 28s 13ms/step - loss: 1.6929e-04 - val_loss: 1.9921e-04
Epoch 11/40
2188/2188 ━━━━━━━━━━━━

I0000 00:00:1750345990.196814      69 devices.cc:67] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
I0000 00:00:1750345990.197036      69 single_machine.cc:361] Starting new session
I0000 00:00:1750345991.012847      69 devices.cc:67] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
I0000 00:00:1750345991.013064      69 single_machine.cc:361] Starting new session


✅ Modelo GRU_model_mse.onnx entrenado y guardado correctamente.
Scaler GRU guardado.
Historial de entrenamiento GRU guardado.


In [12]:
px.line(y=history.history['loss'], title='Pérdida durante el entrenamiento').show()

In [13]:
# Configuración de hiperparámetros
model_type = 'SimpleRNN'  # Cambia por 'SimpleRNN', 'LSTM' o 'GRU' para entrenar otros modelos
loss_function = 'mae'  # Puedes cambiar por 'mse' o 'mae'
n_pasos = 24
n_epochs = 50
batch_size = 32
unidades = 50

# Crear secuencias
X_train, y_train = crear_secuencias(df_train['value_scaled'].values, n_pasos)
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))

# Crear secuencias para Test
X_test, y_test = crear_secuencias(df_test['value_scaled'].values, n_pasos)
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

# Definir el modelo
if model_type == 'SimpleRNN':
    rnn_layer = SimpleRNN(unidades, activation='tanh', input_shape=(n_pasos, 1))
elif model_type == 'LSTM':
    rnn_layer = LSTM(unidades, activation='tanh', input_shape=(n_pasos, 1))
elif model_type == 'GRU':
    rnn_layer = GRU(unidades, activation='tanh', input_shape=(n_pasos, 1))

model = Sequential([
    rnn_layer,
    Dense(1)
])

model.compile(optimizer='adam', loss=loss_function)

# Entrenamiento
history = model.fit(
    X_train, y_train,
    epochs=n_epochs,
    batch_size=batch_size,
    validation_data=(X_test, y_test),  # ⚙️ Esto es obligatorio para tener val_loss
    verbose=1
)

# Guardar el scaler
#model.save(f'{model_type}_model_{loss_function}.keras')

with open(f'scaler_{model_type}_{loss_function}.pkl', 'wb') as f:
    pickle.dump(scaler, f)
    
# Guardar el history
with open(f"{model_type}_history_{loss_function}.pkl", 'wb') as f:
    pickle.dump(history.history, f)

# Guardar el modelo en ONNX
# 1. Crear el input_signature
input_signature = [tf.TensorSpec([None, n_pasos, 1], tf.float32, name="input")]

# 2. Decorar la función (no obtener concrete_func)
@tf.function(input_signature=input_signature)
def model_func(x):
    return model(x)

# 3. Convertir a ONNX directamente desde la función decorada
onnx_model, _ = tf2onnx.convert.from_function(
    model_func,  # 👉 Aquí debes pasar la función decorada, no la concrete_func
    input_signature=input_signature,
    opset=13,
    output_path=f'{model_type}_model_{loss_function}.onnx'
)

print(f'✅ Modelo {model_type}_model_{loss_function}.onnx entrenado y guardado correctamente.')
print(f'Scaler {model_type} guardado.')
print(f"Historial de entrenamiento {model_type} guardado.")

Epoch 1/50


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



2188/2188 ━━━━━━━━━━━━━━━━━━━━ 16s 6ms/step - loss: 0.0280 - val_loss: 0.0101
Epoch 2/50
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 14s 6ms/step - loss: 0.0107 - val_loss: 0.0086
Epoch 3/50
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 14s 6ms/step - loss: 0.0103 - val_loss: 0.0086
Epoch 4/50
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 14s 6ms/step - loss: 0.0101 - val_loss: 0.0093
Epoch 5/50
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 14s 6ms/step - loss: 0.0100 - val_loss: 0.0090
Epoch 6/50
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 14s 6ms/step - loss: 0.0098 - val_loss: 0.0104
Epoch 7/50
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 14s 6ms/step - loss: 0.0098 - val_loss: 0.0086
Epoch 8/50
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 13s 6ms/step - loss: 0.0096 - val_loss: 0.0091
Epoch 9/50
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 14s 6ms/step - loss: 0.0096 - val_loss: 0.0085
Epoch 10/50
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 14s 6ms/step - loss: 0.0095 - val_loss: 0.0080
Epoch 11/50
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 14s 6ms/step - loss: 0.0095 - val_loss: 0.0082
Epoch 12/50
2188/2188 ━━━━━━━━

I0000 00:00:1750346679.644343      69 devices.cc:67] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
I0000 00:00:1750346679.644590      69 single_machine.cc:361] Starting new session
I0000 00:00:1750346679.853884      69 devices.cc:67] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
I0000 00:00:1750346679.854099      69 single_machine.cc:361] Starting new session


✅ Modelo SimpleRNN_model_mae.onnx entrenado y guardado correctamente.
Scaler SimpleRNN guardado.
Historial de entrenamiento SimpleRNN guardado.


In [14]:
px.line(y=history.history['loss'], title='Pérdida durante el entrenamiento').show()

In [15]:
# Configuración de hiperparámetros
model_type = 'LSTM'  # Cambia por 'SimpleRNN', 'LSTM' o 'GRU' para entrenar otros modelos
loss_function = 'mae'  # Puedes cambiar por 'mse' o 'mae'
n_pasos = 24
n_epochs = 50
batch_size = 32
unidades = 50

# Crear secuencias
X_train, y_train = crear_secuencias(df_train['value_scaled'].values, n_pasos)
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))

# Crear secuencias para Test
X_test, y_test = crear_secuencias(df_test['value_scaled'].values, n_pasos)
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

# Definir el modelo
if model_type == 'SimpleRNN':
    rnn_layer = SimpleRNN(unidades, activation='tanh', input_shape=(n_pasos, 1))
elif model_type == 'LSTM':
    rnn_layer = LSTM(unidades, activation='tanh', input_shape=(n_pasos, 1))
elif model_type == 'GRU':
    rnn_layer = GRU(unidades, activation='tanh', input_shape=(n_pasos, 1))

model = Sequential([
    rnn_layer,
    Dense(1)
])

model.compile(optimizer='adam', loss=loss_function)

# Entrenamiento
history = model.fit(
    X_train, y_train,
    epochs=n_epochs,
    batch_size=batch_size,
    validation_data=(X_test, y_test),  # ⚙️ Esto es obligatorio para tener val_loss
    verbose=1
)

# Guardar el scaler
#model.save(f'{model_type}_model_{loss_function}.keras')

with open(f'scaler_{model_type}_{loss_function}.pkl', 'wb') as f:
    pickle.dump(scaler, f)
    
# Guardar el history
with open(f"{model_type}_history_{loss_function}.pkl", 'wb') as f:
    pickle.dump(history.history, f)

# Guardar el modelo en ONNX
# 1. Crear el input_signature
input_signature = [tf.TensorSpec([None, n_pasos, 1], tf.float32, name="input")]

# 2. Decorar la función (no obtener concrete_func)
@tf.function(input_signature=input_signature)
def model_func(x):
    return model(x)

# 3. Convertir a ONNX directamente desde la función decorada
onnx_model, _ = tf2onnx.convert.from_function(
    model_func,  # 👉 Aquí debes pasar la función decorada, no la concrete_func
    input_signature=input_signature,
    opset=13,
    output_path=f'{model_type}_model_{loss_function}.onnx'
)

print(f'✅ Modelo {model_type}_model_{loss_function}.onnx entrenado y guardado correctamente.')
print(f'Scaler {model_type} guardado.')
print(f"Historial de entrenamiento {model_type} guardado.")

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



Epoch 1/50
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 29s 12ms/step - loss: 0.0488 - val_loss: 0.0171
Epoch 2/50
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 26s 12ms/step - loss: 0.0136 - val_loss: 0.0137
Epoch 3/50
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 26s 12ms/step - loss: 0.0121 - val_loss: 0.0107
Epoch 4/50
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 26s 12ms/step - loss: 0.0113 - val_loss: 0.0105
Epoch 5/50
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 26s 12ms/step - loss: 0.0109 - val_loss: 0.0117
Epoch 6/50
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 26s 12ms/step - loss: 0.0105 - val_loss: 0.0101
Epoch 7/50
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 26s 12ms/step - loss: 0.0099 - val_loss: 0.0101
Epoch 8/50
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 26s 12ms/step - loss: 0.0094 - val_loss: 0.0103
Epoch 9/50
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 26s 12ms/step - loss: 0.0093 - val_loss: 0.0099
Epoch 10/50
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 26s 12ms/step - loss: 0.0090 - val_loss: 0.0097
Epoch 11/50
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 26s 12ms/step - loss: 0.0088 - val_loss: 0.0090
Epoch 12

I0000 00:00:1750347985.374255      69 devices.cc:67] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
I0000 00:00:1750347985.374473      69 single_machine.cc:361] Starting new session
I0000 00:00:1750347985.597658      69 devices.cc:67] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
I0000 00:00:1750347985.597853      69 single_machine.cc:361] Starting new session


✅ Modelo LSTM_model_mae.onnx entrenado y guardado correctamente.
Scaler LSTM guardado.
Historial de entrenamiento LSTM guardado.


In [16]:
px.line(y=history.history['loss'], title='Pérdida durante el entrenamiento').show()

In [17]:
# Configuración de hiperparámetros
model_type = 'GRU'  # Cambia por 'SimpleRNN', 'LSTM' o 'GRU' para entrenar otros modelos
loss_function = 'mae'  # Puedes cambiar por 'mse' o 'mae'
n_pasos = 24
n_epochs = 50
batch_size = 32
unidades = 50

# Crear secuencias
X_train, y_train = crear_secuencias(df_train['value_scaled'].values, n_pasos)
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))

# Crear secuencias para Test
X_test, y_test = crear_secuencias(df_test['value_scaled'].values, n_pasos)
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

# Definir el modelo
if model_type == 'SimpleRNN':
    rnn_layer = SimpleRNN(unidades, activation='tanh', input_shape=(n_pasos, 1))
elif model_type == 'LSTM':
    rnn_layer = LSTM(unidades, activation='tanh', input_shape=(n_pasos, 1))
elif model_type == 'GRU':
    rnn_layer = GRU(unidades, activation='tanh', input_shape=(n_pasos, 1))

model = Sequential([
    rnn_layer,
    Dense(1)
])

model.compile(optimizer='adam', loss=loss_function)

# Entrenamiento
history = model.fit(
    X_train, y_train,
    epochs=n_epochs,
    batch_size=batch_size,
    validation_data=(X_test, y_test),  # ⚙️ Esto es obligatorio para tener val_loss
    verbose=1
)

# Guardar el scaler
#model.save(f'{model_type}_model_{loss_function}.keras')

with open(f'scaler_{model_type}_{loss_function}.pkl', 'wb') as f:
    pickle.dump(scaler, f)
    
# Guardar el history
with open(f"{model_type}_history_{loss_function}.pkl", 'wb') as f:
    pickle.dump(history.history, f)

# Guardar el modelo en ONNX
# 1. Crear el input_signature
input_signature = [tf.TensorSpec([None, n_pasos, 1], tf.float32, name="input")]

# 2. Decorar la función (no obtener concrete_func)
@tf.function(input_signature=input_signature)
def model_func(x):
    return model(x)

# 3. Convertir a ONNX directamente desde la función decorada
onnx_model, _ = tf2onnx.convert.from_function(
    model_func,  # 👉 Aquí debes pasar la función decorada, no la concrete_func
    input_signature=input_signature,
    opset=13,
    output_path=f'{model_type}_model_{loss_function}.onnx'
)

print(f'✅ Modelo {model_type}_model_{loss_function}.onnx entrenado y guardado correctamente.')
print(f'Scaler {model_type} guardado.')
print(f"Historial de entrenamiento {model_type} guardado.")

Epoch 1/50


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



2188/2188 ━━━━━━━━━━━━━━━━━━━━ 32s 13ms/step - loss: 0.0509 - val_loss: 0.0151
Epoch 2/50
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 29s 13ms/step - loss: 0.0132 - val_loss: 0.0111
Epoch 3/50
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 29s 13ms/step - loss: 0.0123 - val_loss: 0.0103
Epoch 4/50
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 29s 13ms/step - loss: 0.0114 - val_loss: 0.0091
Epoch 5/50
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 29s 13ms/step - loss: 0.0105 - val_loss: 0.0089
Epoch 6/50
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 29s 13ms/step - loss: 0.0103 - val_loss: 0.0094
Epoch 7/50
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 29s 13ms/step - loss: 0.0098 - val_loss: 0.0093
Epoch 8/50
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 29s 13ms/step - loss: 0.0096 - val_loss: 0.0081
Epoch 9/50
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 29s 13ms/step - loss: 0.0095 - val_loss: 0.0100
Epoch 10/50
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 29s 13ms/step - loss: 0.0093 - val_loss: 0.0089
Epoch 11/50
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 29s 13ms/step - loss: 0.0091 - val_loss: 0.0088
Epoch 12/50
2188/21

I0000 00:00:1750349424.442013      69 devices.cc:67] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
I0000 00:00:1750349424.442218      69 single_machine.cc:361] Starting new session
I0000 00:00:1750349424.694198      69 devices.cc:67] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
I0000 00:00:1750349424.694389      69 single_machine.cc:361] Starting new session


✅ Modelo GRU_model_mae.onnx entrenado y guardado correctamente.
Scaler GRU guardado.
Historial de entrenamiento GRU guardado.


In [18]:
px.line(y=history.history['loss'], title='Pérdida durante el entrenamiento').show()

In [19]:
# Cargar datos para Prophet
df_prophet = get_demanda_data()
df_prophet['datetime'] = df_prophet['datetime'].dt.tz_localize(None)
df_prophet = df_prophet.rename(columns={"datetime": "ds", "value": "y"})

In [20]:
# Guardar en csv 
df_prophet.to_csv('datos_prediccion_prophet.csv', index=False)

In [21]:
# Definir granularidades
granularidades = {
    'D': 'diaria',
    'W': 'semanal',
    'ME': 'mensual',
    'QE': 'trimestral',
    '2Q': 'semestral',
    'YE': 'anual'
}

# Entrenar y guardar un modelo por granularidad
for freq, nombre in granularidades.items():
    print(f'Entrenando modelo para granularidad: {nombre}')
    
    # Reagrupar datos según granularidad
    df_freq = df_prophet.copy()
    df_freq = df_freq.groupby(pd.Grouper(key='ds', freq=freq)).sum().reset_index()

    model_prophet = Prophet()
    model_prophet.fit(df_freq)

    # Guardar modelo
    joblib.dump(model_prophet, f'prophet_model_{nombre}.joblib')
    print(f'✅ Modelo Prophet {nombre} guardado correctamente.')

print('🎉 Todos los modelos entrenados y guardados.')

Entrenando modelo para granularidad: diaria


16:10:46 - cmdstanpy - INFO - Chain [1] start processing
16:10:47 - cmdstanpy - INFO - Chain [1] done processing
16:10:47 - cmdstanpy - INFO - Chain [1] start processing
16:10:47 - cmdstanpy - INFO - Chain [1] done processing
16:10:47 - cmdstanpy - INFO - Chain [1] start processing
16:10:48 - cmdstanpy - INFO - Chain [1] done processing
16:10:48 - cmdstanpy - INFO - Chain [1] start processing


✅ Modelo Prophet diaria guardado correctamente.
Entrenando modelo para granularidad: semanal
✅ Modelo Prophet semanal guardado correctamente.
Entrenando modelo para granularidad: mensual
✅ Modelo Prophet mensual guardado correctamente.
Entrenando modelo para granularidad: trimestral


16:10:48 - cmdstanpy - INFO - Chain [1] done processing
/tmp/ipykernel_69/1031065477.py:17: FutureWarning:

'Q' is deprecated and will be removed in a future version, please use 'QE' instead.

16:10:48 - cmdstanpy - INFO - Chain [1] start processing
16:10:48 - cmdstanpy - INFO - Chain [1] done processing
16:10:48 - cmdstanpy - INFO - Chain [1] start processing
16:10:48 - cmdstanpy - INFO - Chain [1] done processing


✅ Modelo Prophet trimestral guardado correctamente.
Entrenando modelo para granularidad: semestral
✅ Modelo Prophet semestral guardado correctamente.
Entrenando modelo para granularidad: anual
✅ Modelo Prophet anual guardado correctamente.
🎉 Todos los modelos entrenados y guardados.
